In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('dataset.csv')  # Replace with your dataset path
print(df.head())  # View the first few rows


                                                 url  length_url  \
0              http://www.crestonwood.com/router.php          37   
1  http://shadetreetechnology.com/V4/validation/a...          77   
2  https://support-appleld.com.secureupdate.duila...         126   
3                                 http://rgipt.ac.in          18   
4  http://www.iracing.com/tracks/gateway-motorspo...          55   

   length_hostname  ip  nb_dots  nb_hyphens  nb_at  nb_qm  nb_and  nb_or  ...  \
0               19   0        3           0      0      0       0      0  ...   
1               23   1        1           0      0      0       0      0  ...   
2               50   1        4           1      0      1       2      0  ...   
3               11   0        2           0      0      0       0      0  ...   
4               15   0        2           2      0      0       0      0  ...   

   domain_in_title  domain_with_copyright  whois_registered_domain  \
0                0                

# Set legitimate web as 0, and phish web as 1

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['status'] = le.fit_transform(df['status'])
df.head(3)

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,0
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,1
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,1


split the dataset into training and testing data

In [4]:
from sklearn.model_selection import train_test_split

y = df['status']
X = df.drop(['url', 'status'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# BaseLine model

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7891513560804899
              precision    recall  f1-score   support

           0       0.79      0.80      0.79      1157
           1       0.79      0.78      0.78      1129

    accuracy                           0.79      2286
   macro avg       0.79      0.79      0.79      2286
weighted avg       0.79      0.79      0.79      2286



/Users/kiwi/Desktop/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier(max_depth=5)  # Adjust max_depth based on data complexity
DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9352580927384077
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1157
           1       0.95      0.92      0.93      1129

    accuracy                           0.94      2286
   macro avg       0.94      0.94      0.94      2286
weighted avg       0.94      0.94      0.94      2286



# Advanced Models

In [14]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9615048118985127
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1157
           1       0.96      0.96      0.96      1129

    accuracy                           0.96      2286
   macro avg       0.96      0.96      0.96      2286
weighted avg       0.96      0.96      0.96      2286



In [15]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
MLP.fit(X_train, y_train)
y_pred = MLP.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6924759405074365
              precision    recall  f1-score   support

           0       0.93      0.43      0.58      1157
           1       0.62      0.97      0.76      1129

    accuracy                           0.69      2286
   macro avg       0.77      0.70      0.67      2286
weighted avg       0.78      0.69      0.67      2286



In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Xtrain_lstm = np.array(X).reshape((X.shape[0], 1, X.shape[1]))

lstm = Sequential()
lstm.add(Input(shape=(Xtrain_lstm.shape[1], Xtrain_lstm.shape[2])))
lstm.add(LSTM(units=128))
lstm.add(Dense(1, activation='sigmoid'))
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
lstm.summary()
X_train, X_test, y_train, y_test = train_test_split(Xtrain_lstm, y, train_size=0.8, random_state=42)
print(X_train.shape)
print(y_train.shape)
lstm.fit(X_train, y_train, verbose=0, epochs=10, batch_size=32, validation_split=0.2)
loss, accuracy, precision, recall = lstm.evaluate(X_test, y_test, verbose=0)
y_pred = lstm.predict(X_test, batch_size=64, verbose=0)
y_pred_bool = (y_pred > 0.5).astype(int)
f1 = f1_score(y_test, y_pred_bool)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 128)            │       110,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,721 (432.50 KB)

 Trainable params: 110,721 (432.50 KB)

 Non-trainable params: 0 (0.00 B)

(9144, 1, 87)
(9144,)
Loss: 0.44302308559417725
Accuracy: 0.7869641184806824
Precision: 0.9246031641960144
Recall: 0.6191319823265076
F1-Score: 0.7416445623342175
